# Typesafe plotting?
One idea of how to "avoid writing a vega spec in mutable JSON" was to see if we could make this "typesafe". Somehow. If you had a bunch of case classes which followed the vega JSON structure, then you'd get tab completion, editor support etc. Maybe this would dodge the "having to learn vega" problem.

It turns out, that vega publish a [schema](https://github.com/vega/schema). What you'd now need, is a tool which [turned JSON schema into case classes](https://quicktype.io) along with someone nutty enough to fork it, [implement dodgy, experimental scala 3 support](https://github.com/Quafadas/quicktype/blob/scala3/src/quicktype-core/language/Scala3.ts) and point it at the vega schema. Then you'd get some [code]() and [javadoc]([![javadoc](https://javadoc.io/badge2/io.github.quafadas/dedav4s_3/javadoc.svg)](https://javadoc.io/doc/io.github.quafadas/dedav4s_3)) along these lines, and could try this;

In [1]:
import $ivy.`io.github.quafadas::dedav4s:0.9-9531ccc-20230523T195316Z-SNAPSHOT`
import $ivy.`io.circe::circe-core:0.14.5`

import viz.PlotTargets.almond
import viz.dsl.vegaLite.*
import viz.dsl.DslPlot.*
import io.circe.*
import io.circe.parser.*
import io.circe.syntax.*
import cats.syntax.all.*
import viz.vega.plots.SpecUrl

case class LinePlottable(x:Double, y:Double) 

def toPlotDataset(p: Seq[LinePlottable]): URLData =
  val inlineDataset : InlineDataset = for(p1 <- p) yield {        
    Map("x"-> Encoder.encodeDouble(p1.x).some, "y" -> Encoder.encodeDouble(p1.y).some)
  }
  URLData(
    values = inlineDataset.some
  )
end toPlotDataset

val somePlot : VegaLiteDsl = decode[VegaLiteDsl]( viz.vega.plots.SpecUrl.LineChartLite.jsonSpec.toString()).fold(error => throw new Exception(error), identity)

//somePlot.plot



import $ivy.$                                                                  

import $ivy.$                            


import viz.PlotTargets.almond

import viz.dsl.vegaLite.*

import viz.dsl.DslPlot.*

import io.circe.*

import io.circe.parser.*

import io.circe.syntax.*

import cats.syntax.all.*

import viz.vega.plots.SpecUrl


defined class LinePlottable
defined function toPlotDataset
somePlot: VegaLiteDsl = VegaLiteDsl(
  $schema = Some(value = "https://vega.github.io/schema/vega-lite/v5.json"),
  align = None,
  autosize = None,
  background = None,
  bounds = None,
  center = None,
  config = None,
  data = Some(
    value = URLData(
      format = None,
      name = None,
      url = Some(value = "data/stocks.csv"),
      values = None,
      sequence = None,
      sphere = None,
      graticule = None
    )
  ),
  datasets = None,
  description = Some(value = "Google's stock price over time."),
  encoding = Some(
    value = EdEncoding(
      angle = None,
      color = N

This is nominally "typesafe". You get tab completion, and the compiler will tell you if you've made a mistake. But it's not _that_ typesafe - it's pure coincidence, that the case class happens to have the x,y fields, which are needed, by this particular plot. 

In the end, `Map[String, Double]` isnt _significantly_ better than the mutable alternative earlier. Definitely more headspace to get there, though. 